<a href="https://colab.research.google.com/github/HaaLeo/vague-requirements-scripts/blob/master/colab-notebooks/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classify requirements as vague or not using [ktrain](https://github.com/amaiya/ktrain) and tensorflow


## Install dependencies
*ktrain* requires TensorFlow 2.1. See [amaiya/ktrain#151](https://github.com/amaiya/ktrain/issues/151).
Further we install a forked version of eli5lib to gain insights in the model's decision process and some self built helper functions to preprocess MTurk result files.

In [2]:
!pip3 install -q tensorflow_gpu==2.1.0 ktrain==0.17.3
!pip3 install -q -U git+https://github.com/HaaLeo/vague-requirements-scripts
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

     |████████████████████████████████| 421.8MB 42kB/s 
     |████████████████████████████████| 25.2MB 127kB/s 
     |████████████████████████████████| 3.9MB 40.8MB/s 
     |████████████████████████████████| 450kB 43.7MB/s 
     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 6.7MB 41.2MB/s 
     |████████████████████████████████| 983kB 37.7MB/s 
     |████████████████████████████████| 245kB 43.9MB/s 
     |████████████████████████████████| 757kB 36.0MB/s 
     |████████████████████████████████| 471kB 42.8MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
     |████████████████████████████████| 3.0MB 36.3MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


Check versions and enable logging

In [3]:
import tensorflow as tf
import ktrain
assert tf.__version__ == '2.1.0'
assert ktrain.__version__ == '0.17.3'

import logging
import sys

logging.basicConfig(
    format='%(asctime)s [%(threadName)-20.20s] [%(levelname)-5.5s]  %(message)s',
    stream=sys.stdout,
    level=logging.DEBUG)

## Set Parameters
Set the parameters for this run.
Ktrain ignores `max_features` and `ngram_range` in v0.17.3, see [amaiya/ktrain/issues#190](https://github.com/amaiya/ktrain/issues/190)

In [4]:
indices_to_read = [0,2,3,4] # indicate which MTurk files shall be read.
DATA_FILE_NAMES = [f'corpus-batch-{i}-mturk.csv' for i in indices_to_read]

RANDOM_STATE = 1 # for seeding

LEARNING_RATE = 5e-5
EPOCHS = 4
MODEL_NAME = 'distilbert-base-uncased'
MAX_LEN = 512
BATCH_SIZE = 6
MAX_FEATURES = 35_000
NGRAM_RANGE = 1

CLASS_NAMES = ['not-vague', 'vague'] # 0=not-vague 1=vague

PREPROCESS_MODE = 'distilbert'

## Load Dataset

### Mount Google Drive
Mount the google drive to access the dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Load Dataset Into Arrays

In [6]:
from vaguerequirementslib import read_csv_files, build_confusion_matrix, calc_majority_label
import pandas as pd

def read_drive_data(files_list: list, separator: str) -> pd.DataFrame:
    """
    Calculate the majority label for the given source file list

    Args:
        files_list (list): The CSV files to calculate the majority label for
        separator (str): The CSV separator
        drop_ties (bool): If there is a tie in votes (e.g.: One votes for vague one for not vague) then drop this entry from the confusion matrix.

    Returns:
        pd.DataFrame: The dataframe containing the majority label.
    """
    df = read_csv_files(files_list, separator)
    confusion_matrix = build_confusion_matrix(df, drop_ties=True)
    return calc_majority_label(confusion_matrix)

# Read all data
df = read_drive_data(
    [f'/content/drive/My Drive/datasets/corpus/labeled/{file_name}' for file_name in DATA_FILE_NAMES],
    ','
  )
df.head()

2020-07-06 14:08:28,308 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-0-mturk.csv" with 200 rows.
2020-07-06 14:08:28,702 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-2-mturk.csv" with 194 rows.
2020-07-06 14:08:29,110 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-3-mturk.csv" with 198 rows.
2020-07-06 14:08:29,519 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-4-mturk.csv" with 196 rows.
2020-07-06 14:08:29,539 [MainThread          ] [INFO ]  Build confusion matrix.
2020-07-06 14:08:29,685 [MainThread          ] [INFO ]  Dropped 180 requirements due to ties.
2020-07-06 14:08:29,687 [MainThread          ] [INFO ]  Built confusion matrix including 214 of 394 requirements. 
2020-07-06 14:08:29,690 [MainThread          ] [INFO ]  Overall "vague" votes count = 9

,requirement,vague_count,not_vague_count,majority_label
0,A fallback per band feature set resulting from...,2,0,1
1,Actuation of steering shall be possible regard...,0,2,0
2,"Additionally, the ZigBee end device shall then...",0,2,0
3,"Additionally, the plan provides traceability f...",2,0,1
4,"After completion of release of the resources, ...",0,2,0


### Split data set


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from typing import Tuple, List
from ktrain import text as txt

def upsample(data_frame: pd.DataFrame) -> pd.DataFrame:
    """
    Up sample the given dataframe to contain equally much samples of vague and not-vague requirements.

    Args:
        data_frame (pd.DataFrame): The data frame to upsample.

    Returns:
        pd.DataFrame: The upsampled data frame.
    """
    vague_df = data_frame[data_frame.majority_label == 1]
    not_vague_df = data_frame[data_frame.majority_label == 0]

    if int(vague_df.majority_label.value_counts()) > int(not_vague_df.majority_label.value_counts()):
        df_to_upsample = not_vague_df
        majority_df = vague_df
    else:
        df_to_upsample = vague_df
        majority_df = not_vague_df

    upsampled_df = resample(
        df_to_upsample, 
        replace=True,     # sample with replacement
        n_samples=majority_df.shape[0],    # to match majority class
        random_state=RANDOM_STATE) # reproducible results
    result = pd.concat([upsampled_df, majority_df])

    print(f'Upsampled dataset: vague count="{result.sum()["majority_label"]}", not vague count="{result.shape[0] - result.sum()["majority_label"]}"')

    return result


def split_dataset(data_frame: pd.DataFrame) -> Tuple[List[str], List[int], List[str], List[int], List[str], List[int]]:
    """
    Split the dataset into training, validation and test set.

    Args:
        data_frame (pd.DataFrame): The data frame to generate the data sets from.

    Returns:
        Tuple[List[str], List[int], List[str], List[int], List[str], List[int]]: x_train, y_train, x_val, y_val, x_test, y_test
    """
    train_df, val_test_df = train_test_split(data_frame, test_size=0.2, random_state=RANDOM_STATE, stratify=data_frame['majority_label'])
    val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=RANDOM_STATE, stratify=val_test_df['majority_label'])

    print(f'Training dataset: vague count="{train_df.sum()["majority_label"]}", not vague count="{train_df.shape[0] - train_df.sum()["majority_label"]}"')
    print(f'Validation dataset: vague count="{val_df.sum()["majority_label"]}", not vague count="{val_df.shape[0] - val_df.sum()["majority_label"]}"')
    print(f'Test dataset: vague count="{test_df.sum()["majority_label"]}", not vague count="{test_df.shape[0] - test_df.sum()["majority_label"]}"')

    return train_df, val_df, test_df


def preprocess_data(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame) -> Tuple:
    def _preprocess(my_df: pd.DataFrame) -> Tuple:
        dummy_df = pd.DataFrame.from_dict({'requirement': ['foo', 'bar'], 'majority_label': [0, 1]})
        return txt.texts_from_df(my_df, text_column='requirement', label_columns=['majority_label'], val_df=dummy_df,  max_features=MAX_FEATURES, maxlen=MAX_LEN,  ngram_range=NGRAM_RANGE, preprocess_mode=PREPROCESS_MODE, random_state=RANDOM_STATE)
    
    train_data, _, _ = _preprocess(train_df)
    val_data, _, _ = _preprocess(val_df)
    test_data, _, test_preproc = _preprocess(test_df)

    return train_data, val_data, test_data, test_preproc

# Split the data set
train_df, val_df, test_df = split_dataset(df)

# Upsample train_df
train_df = upsample(train_df)

# Preprocess for Transfer Learning
train_data, val_data, test_data, test_preproc = preprocess_data(train_df, val_df, test_df)


Training dataset: vague count="38", not vague count="133"
Validation dataset: vague count="5", not vague count="16"
Test dataset: vague count="5", not vague count="17"
Upsampled dataset: vague count="133", not vague count="133"
preprocessing train...
language: en
train sequence lengths:
	mean : 22
	95percentile : 40
	99percentile : 59
2020-07-06 14:24:36,701 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-06 14:24:37,103 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1
	95percentile : 1
	99percentile : 1


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:229: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  'If this is a sentence pair classification task, please cast to tuple.')


preprocessing train...
language: en
train sequence lengths:
	mean : 22
	95percentile : 38
	99percentile : 46
2020-07-06 14:24:37,497 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-06 14:24:37,896 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1
	95percentile : 1
	99percentile : 1


preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 36
	99percentile : 38
2020-07-06 14:24:38,093 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-06 14:24:38,480 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1
	95percentile : 1
	99percentile : 1


## STEP 1:  Create a Transformer Model and Train it

We will use [DistilBERT](https://arxiv.org/abs/1910.01108).

In [7]:
from datetime import datetime
from pytz import timezone

# Create the transformer
t = txt.Transformer(MODEL_NAME, maxlen=MAX_LEN, class_names=CLASS_NAMES)
t.preprocess_train_called = True # Simulate call to preprocess_train()

# Get the model and learner
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=BATCH_SIZE)

# For every triggered fitting run create a new directory where the results will be saved
now = datetime.now(timezone('Europe/Berlin'))
result_dir = f'/content/drive/My Drive/runs/{now.strftime("%Y-%m-%d/%H-%M-%S")}'

# Fit the model
learner.fit_onecycle(LEARNING_RATE, EPOCHS)

2020-07-06 11:18:35,048 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-06 11:18:35,444 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/distilbert-base-uncased-config.json HTTP/1.1" 200 0
2020-07-06 11:18:35,448 [MainThread          ] [DEBUG]  Attempting to acquire lock 139847894703744 on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
2020-07-06 11:18:35,449 [MainThread          ] [INFO ]  Lock 139847894703744 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
2020-07-06 11:18:35,452 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-06 11:18:35,851 [MainThread          ] [DEBUG]  https://s3.amazonaws.com


2020-07-06 11:18:35,892 [MainThread          ] [DEBUG]  Attempting to release lock 139847894703744 on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
2020-07-06 11:18:35,893 [MainThread          ] [INFO ]  Lock 139847894703744 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
2020-07-06 11:18:35,897 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): cdn.huggingface.co:443
2020-07-06 11:18:35,985 [MainThread          ] [DEBUG]  https://cdn.huggingface.co:443 "HEAD /distilbert-base-uncased-tf_model.h5 HTTP/1.1" 200 0
2020-07-06 11:18:35,989 [MainThread          ] [DEBUG]  Attempting to acquire lock 139850769762736 on /root/.cache/torch/transformers/cce28882467f298a29fc905b9dd1683695d96198a83432fe707089dccd71c019.e02bd57e9d85078


2020-07-06 11:18:41,411 [MainThread          ] [DEBUG]  Attempting to release lock 139850769762736 on /root/.cache/torch/transformers/cce28882467f298a29fc905b9dd1683695d96198a83432fe707089dccd71c019.e02bd57e9d8507853eccc7c04ac2e938a6cdaff4b9bf941c10e781b61ddb9bbd.h5.lock
2020-07-06 11:18:41,412 [MainThread          ] [INFO ]  Lock 139850769762736 released on /root/.cache/torch/transformers/cce28882467f298a29fc905b9dd1683695d96198a83432fe707089dccd71c019.e02bd57e9d8507853eccc7c04ac2e938a6cdaff4b9bf941c10e781b61ddb9bbd.h5.lock


begin training using onecycle policy with max lr of 5e-05...
Train for 29 steps, validate for 1 steps
Epoch 1/4
29/29 [==============================] - 23s 776ms/step - loss: 0.5924 - accuracy: 0.7836 - val_loss: 0.5647 - val_accuracy: 0.7619
Epoch 2/4
29/29 [==============================] - 15s 506ms/step - loss: 0.5142 - accuracy: 0.7778 - val_loss: 0.5442 - val_accuracy: 0.7619
Epoch 3/4
29/29 [==============================] - 15s 506ms/step - loss: 0.4545

## STEP 2: Evaluate the model
Evaluate the model using the `test_data`.

In [8]:
test_result = learner.validate(class_names=t.get_classes(), val_data=test_data)
print(test_result)

              precision    recall  f1-score   support

   not-vague       0.78      0.82      0.80        17
       vague       0.25      0.20      0.22         5

    accuracy                           0.68        22
   macro avg       0.51      0.51      0.51        22
weighted avg       0.66      0.68      0.67        22

[[14  3]
 [ 4  1]]


## STEP 3: Gather Results

Gather results, calulate metrics and write them to the drive.

In [9]:
import os
import json
from os import path

from vaguerequirementslib import TP, TN, FP, FN, calc_all_metrics

def build_result_data(test_result: List) -> dict:
    result_data = {
        'metrics':{
            'vague': {
                TP: int(test_result[1][1]),
                FP: int(test_result[0][1]),
                TN: int(test_result[0][0]),
                FN: int(test_result[1][0])
            },
            'not_vague': {
                TP: int(test_result[0][0]),
                FP: int(test_result[1][0]),
                TN: int(test_result[1][1]),
                FN: int(test_result[0][1])
            }
        },
        'misc': {   
            'data_files': DATA_FILE_NAMES,
            'random_state': RANDOM_STATE
        },
        'hyperparameter': {
            'learning_rate': LEARNING_RATE,
            'epochs': EPOCHS,
            'model_name': MODEL_NAME,
            'max_len': MAX_LEN,
            'batch_size': BATCH_SIZE,
            'max_features': MAX_FEATURES,
            'ngram_range': NGRAM_RANGE
        }
    }
    result_data['metrics']['not_vague'].update(calc_all_metrics(**result_data['metrics']['not_vague']))
    result_data['metrics']['vague'].update(calc_all_metrics(**result_data['metrics']['vague']))
    
    return result_data

result_data = build_result_data(test_result)
# Get the predictor
predictor = ktrain.get_predictor(learner.model, preproc=t)

## STEP 3.1 Save the Results
Check out the [FAQ](https://github.com/amaiya/ktrain/blob/master/FAQ.md#method-1-using-predictor-api-works-for-any-model) for how to load a model from a predictor.

In [1]:
# Save the corresponding model (predictor)
predictor.save(path.join(result_dir, 'predictor'))

# Save the evaluation result (test_data results)
with open(path.join(result_dir, 'evaluation.json'), mode='w', encoding='utf-8') as json_file:
    json.dump(result_data, json_file, indent=4)

NameError: ignored

## STEP 4 Inspect the Model and its Losses

Let's examine the validation example about which we were the most wrong.

In [11]:
learner.view_top_losses(n=4, preproc=t, val_data=test_data)
top_losses = learner.top_losses(n=4, preproc=t, val_data=test_data)

----------
id:0 | loss:2.74 | true:vague | pred:not-vague)

----------
id:20 | loss:2.08 | true:vague | pred:not-vague)

----------
id:4 | loss:1.35 | true:vague | pred:not-vague)

----------
id:6 | loss:1.24 | true:not-vague | pred:vague)



In [13]:
top_loss_req = test_df.iloc[6]['requirement'] # Requirement that produces top loss

print(predictor.predict(top_loss_req))

# predicted probability scores for each category
print(predictor.predict_proba(top_loss_req))
print(top_loss_req)

vague


[0.28983238 0.71016765]
Conformance tests described in clause 5.3.6 shall be carried out.


Let's invoke the `explain` method to see which words contribute most to the classification.

In [14]:
from IPython.core.display import display

for id, _, _, _ in top_losses:
    top_loss_req = test_df.iloc[id]['requirement'] # Requirement that produces top loss
    display(predictor.explain(top_loss_req, n_samples=1_000))

The words in the darkest shade of green contribute most to the classification and agree with what you would expect for this example.